In [1]:
import string
import re
import pandas as pd
import spacy
from spacy.lang.es.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.svm import LinearSVC
import joblib

In [2]:
nlp = spacy.load("es_core_news_sm")

In [46]:
entrenamiento = pd.read_csv("dataset_entrenamiento.csv")
entrenamiento.dropna(inplace=True)

In [13]:
puntuaciones = string.punctuation + "¡¿"
stopwords = set(STOP_WORDS)

def limpieza_de_datos(oracion):
    # Eliminar emojis utilizando expresiones regulares
    oracion = re.sub(r'[^\w\s,]', '', oracion)
    
    doc = nlp(oracion)
    
    tokens_limpios = []
    for token in doc:
        # Ignorar tokens de longitud uno y caracteres no alfabéticos
        if token.is_alpha and len(token.text) > 1:
            # Convertir a minusculas y lematizar sustantivos y adjetivos
            if token.lemma_ != "-PRON-" and token.pos_ in ['NOUN', 'ADJ']:
                temp = token.lemma_.lower().strip()
            else:
                temp = token.lower_
            
            # Filtrar stopwords y puntuaciones
            if temp not in stopwords and temp not in puntuaciones:
                tokens_limpios.append(temp)
    
    return tokens_limpios

In [14]:
tfidf = TfidfVectorizer(tokenizer=limpieza_de_datos, # Se utiliza la funcion de limpieza de datos como tokenizador
                        max_features=1000000,   # Incrementa el número máximo de características
                        ngram_range=(1, 3),     # Incluye trigramas además de unigramas y bigramas
                        max_df=0.4,             # Reduce el umbral para palabras muy frecuentes
                        min_df=5,               # Incrementa el umbral para palabras poco frecuentes
                        use_idf=True,           # Aplica la ponderación IDF
                        smooth_idf=True,        # Suaviza las ponderaciones IDF
                        sublinear_tf=True)      # Aplica escala logarítmica a las frecuencias de términos

# Clasificador SVM lineal
classifier = LinearSVC(C=1.0,                    # Parámetro de regularización
                       class_weight='balanced',  # Ajusta automáticamente los pesos de clase
                       dual=False,               # Usa la formulación primal
                       max_iter=1000000,         # Incrementa el número máximo de iteraciones
                       random_state=42)          # Semilla aleatoria para reproducibilidad

In [15]:
X = entrenamiento["comentario"]
y = entrenamiento["sentimiento"]

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((51726,), (5748,), (51726,), (5748,))

In [17]:
clf = Pipeline([('tfidf', tfidf), ('clf', classifier)])

In [18]:
#evitar que el formato se tome como unknown
y_train = y_train.astype('int')
y_test = y_test.astype('int')

In [ ]:
clf.fit(X_train, y_train)

In [20]:
y_prediccion = clf.predict(X_test)
y_prediccion.shape

(5748,)

In [21]:
print(classification_report(y_test, y_prediccion))

              precision    recall  f1-score   support

           0       0.83      0.81      0.82      1912
           1       0.83      0.80      0.81      1924
           2       0.77      0.82      0.79      1912

    accuracy                           0.81      5748
   macro avg       0.81      0.81      0.81      5748
weighted avg       0.81      0.81      0.81      5748



In [25]:
clf.predict(["Tiene muchas cosas por mejorar"])

array([0])

In [44]:
clf.predict(["muy normal"])

array([2])

In [23]:
joblib.dump(clf, "emotion_clf.pkl")

['emotion_clf.pkl']